In [1]:
import pycountry
import pandas as pd
from IPython.display import Markdown

from tulip.core.collection import TulipCollection
from tulip.data.bloomberg import BloombergClient as bb
from tulip.data.api_haver import HaverApiClient
from tulip.plots import plot_area, plot_lines
from tulip.genai import iris
from tulip.data.gs import GSClient as gs


from tulip_mania.helper_functions.countries import *

ISO_2 = "CN"
pyctry = pycountry.countries.get(alpha_2=ISO_2)

# Clients
hv = HaverApiClient.from_env()


## China
### Economic Forecasts (Goldman Sachs)

In [2]:
from tulip.utils.string_manipulation import c_humanize as humanize

gs_eco_fct = gs.get_eco_forecast(geographyId=ISO_2)

update_time = gs_eco_fct["updateTime"].max()

drop_level = True
only_last_quarter = True


gs_df = (
    gs_eco_fct[~gs_eco_fct.valueCurrency.isnull()]
    .drop(["geographyName", "updateTime", "valueCurrency"], axis=1)
    .replace({"N/A": None})
    .dropna(subset=["forecastValue"])
    .dropna(how="all", axis=1)
)
# Normalize types to avoid TypeError when concatenating strings
gs_df["valueType"] = gs_df["valueType"].astype(str)
# Map seasonality to a string suffix (include comma when present) so concatenation works
gs_df["isSeasonallyAdjusted"] = (
    gs_df["isSeasonallyAdjusted"].map({1: ", SA", 0: ""}).astype(str)
)

if drop_level:
    gs_df = gs_df[gs_df["valueType"] != "Level"]

if only_last_quarter:
    last_quarter_date = gs_df.loc[gs_df["periodType"] == "Quarterly"].index.max()
    not_last_mask = ~(gs_df.index == last_quarter_date) & (
        gs_df["periodType"] == "Quarterly"
    )
    gs_df = gs_df[~not_last_mask]
# Ensure isReal is integer (mapping used later expects 0/1)
gs_df["isReal"] = gs_df["isReal"].astype(int)
metrics_with_multiple_valuetypes = gs_df.groupby(["metricName"])["valueType"].nunique()
metrics_with_multiple_valuetypes = metrics_with_multiple_valuetypes[
    metrics_with_multiple_valuetypes > 1
].index.tolist()
mask = gs_df.metricName.isin(metrics_with_multiple_valuetypes)
gs_df["metricName"] = gs_df.metricName.apply(humanize)

gs_df.loc[mask, "metricName"] = (
    gs_df.loc[mask, "metricName"]
    + gs_df.loc[mask, "isReal"].map({1: " (Real, ", 0: " ("})
    + gs_df.loc[mask, "valueType"]
    + gs_df.loc[mask, "isSeasonallyAdjusted"]
    + ")"
)
gs_df.loc[~mask, "metricName"] = (
    gs_df.loc[~mask, "metricName"]
    + " ("
    + gs_df.loc[~mask, "valueType"]
    + gs_df.loc[~mask, "isSeasonallyAdjusted"]
    + ")"
)


# Format the index date into a readable period string depending on periodType
_dates = gs_df.index.to_series()


def _format_period(ts, period_type):
    if pd.isna(ts):
        return ""
    if period_type == "Yearly":
        return str(ts.year)
    if period_type == "Quarterly":
        q = (ts.month - 1) // 3 + 1
        return f"{ts.year}-Q{q}"
    if period_type == "Monthly":
        return ts.strftime("%Y-%m")
    return ts.strftime("%Y-%m-%d")


gs_df["Period"] = [_format_period(d, p) for d, p in zip(_dates, gs_df["periodType"])]

gs_df = gs_df.drop(
    [
        "valueType",
        "isSeasonallyAdjusted",
        "isReal",
        "geographyId",
        "periodType",
        "isAnnualized",
    ],
    axis=1,
)

gs_df = (
    gs_df.groupby(["Period", "metricName"])
    .last()
    .unstack(
        [
            "Period",
        ],
    )
)

df = gs_df.droplevel(0, axis=1).T


In [4]:
df.iloc[:, 0:7]

metricName,Augmented govt balance (% of GDP),CPI avg (YoY),Consumer expenditure (YoY),Core CPI (YoY),Current account (% of GDP),Domestic demand (YoY),Fixed investment (YoY)
Period,,,,,,,
2025,-12.0,-0.05,4.8,0.71,3.44,3.72,1.5
2025-Q4,NaN,0.17,NaN,0.99,3.59,NaN,NaN


In [5]:
df.iloc[:, 7:14]

metricName,"Gdp (Real, QoQ, SA)","Gdp (Real, YoY)","Gdp (Real, YoY, SA)",Gdp (YoY),Govt balance (% of GDP),Industrial production (YoY),Merchandise exports (YoY)
Period,,,,,,,
2025,NaN,4.96,4.99,3.97,-5.47,5.68,5.54
2025-Q4,4.5,4.48,NaN,3.72,NaN,4.68,3.95


In [6]:
df.iloc[:, 14:]

metricName,Merchandise imports (YoY),"Ppi (QoQ, SA)",Ppi (YoY),"Ppi (YoY, SA)",Total social financing (YoY),Trade balance (% of GDP)
Period,,,,,,
2025,0.18,NaN,-2.64,-2.64,8.5,6.04
2025-Q4,3.84,-0.6,-2.09,NaN,NaN,5.67


### Activity Indicators & Growth Signals
#### Current Activity Indicator (Goldman)

**TODO**: Add GS Current Activity Indicator showing hard vs soft data contributions
- Use `gs.get_CAI_series(geographyId=ISO_2)` with metrics: CAI_HEADLINE, CAI_CONTRIBUTION_TYPE_HARD, CAI_CONTRIBUTION_TYPE_SOFT

In [7]:
cai_series_soft_vs_hard = gs.get_CAI_series(
    geographyId=ISO_2,
    metricName=[
        "CAI_HEADLINE",
        "CAI_CONTRIBUTION_TYPE_HARD",
        "CAI_CONTRIBUTION_TYPE_SOFT",
    ],
    startDate="1980-01-01",
)

cai_series_soft_vs_hard = cai_series_soft_vs_hard.set_index("metricName", append=True)[
    "metricValue"
].unstack("metricName")

cai_series_soft_vs_hard.columns = ["Hard", "Soft", "Headline"]

cai_plot = plot_lines(
    cai_series_soft_vs_hard,
    show_0=True,
    title=f"<b>{pyctry.name} Current Activity Indicator</b> Updated: {pd.Timestamp.today().strftime('%Y-%m-%d')}",
    years_limit=4,
    tick_suffix=" %",
    source="GS",
)

cai_plot

#### PMIs (Manufacturing & Services)

**TODO**: Add China PMI data - both official NBS and Caixin versions
- Manufacturing PMI (NBS and Caixin)
- Services/Non-Manufacturing PMI
- Composite PMI
- Look for Bloomberg tickers like `CPMINDX Index` (NBS Mfg), `ECCNMAN Index` (Caixin Mfg)

In [8]:
pmi_series = {
    "Composite PMI Output": "S924VPMI@EMERGEPR",
    "Manufacturing PMI": "S924VM@EMERGEPR",
    "Manufacturing Employment PMI": "S924VME@EMERGEPR",
    "Nonmanufacturing PMI": "S924VNG@EMERGEPR",
    "Services PMI": "S924VNGS@EMERGEPR",
}

pmi_collection = hv.create_collection(pmi_series)

pmi_response = iris.summarize(pmi_collection, bullets=3)
pmi_response.html()

In [9]:
pmi_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Composite PMI Output,50.0,2025-10-01,50.6,-0.60,-0.16,-0.20,-0.80,2025-11-14 16:26:00
Manufacturing PMI,49.0,2025-10-01,49.8,-0.80,-0.40,0.00,-1.1,2025-11-14 16:26:00
Manufacturing Employment PMI,48.3,2025-10-01,48.5,-0.20,-0.11,0.40,-0.10,2025-11-14 16:26:00
Nonmanufacturing PMI,50.1,2025-10-01,50.0,0.10,0.04,-0.30,-0.10,2025-11-14 16:26:00
Services PMI,50.2,2025-10-01,50.1,0.10,0.03,0.10,0.10,2025-11-14 16:26:00


In [10]:
pmi_collection.dashboard.plots(hlines=50, years_limit=10, source="Haver")

#### Surveys

In [11]:
consumer_confidence_series = {
    "Consumer Confidence": "H924VCC@EMERGEPR",
    "Future Price Expectations": "H924VFP@EMERGEPR",
    "Current Income Sentiment": "H924VCY@EMERGEPR",
    "Future Income Confidence": "H924VFY@EMERGEPR",
    "Current Employment Sentiment": "H924VCL@EMERGEPR",
    "Future Employment Expectations": "H924VCEL@EMERGEPR",
}

consumer_confidence_collection = hv.create_collection(consumer_confidence_series)

business_surveys = hv.create_collection(
    {
        "Industrial Enterprises Macro Performance": "N924VIE@EMERGEPR",
        "Industrial Enterprises Overseas Orders": "N924VIX@EMERGEPR",
        "Logistics Prosperity Index": "N924VTLP@EMERGEPR",
        "Economic Policy Uncertainty (News-Based)": "N924VIUP@EMERGEPR",
        "Trade Policy Uncertainty": "N924VIUT@EMERGEPR",
        "Yicai Chief Economists Confidence": "N924VECI@EMERGEPR",
        "CKGSB Business Conditions Index": "N924VVC@EMERGEPR",
        "Small & Medium Enterprises Development Index": "N924SMD@EMERGEPR",
    }
)

chinese_surveys = consumer_confidence_collection.combine(business_surveys)

chinese_surveys = iris.summarize(chinese_surveys, bullets=3)
chinese_surveys.html()

##### Consumer Surveys

In [12]:
consumer_confidence_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Consumer Confidence,90.3,2025-09-01,90.8,-0.50,-0.24,4.9,4.0,2025-11-14 16:26:00
Future Price Expectations,56.0,2025-07-01,57.3,-1.3,-0.41,-3.3,-3.9,2025-11-14 16:26:00
Current Income Sentiment,47.1,2025-07-01,45.4,1.7,0.79,2.0,0.70,2025-11-14 16:26:00
Future Income Confidence,46.2,2025-07-01,45.5,0.70,0.45,1.0,0.70,2025-11-14 16:26:00
Current Employment Sentiment,26.6,2025-07-01,28.3,-1.7,-0.84,-2.4,-4.5,2025-11-14 16:26:00
Future Employment Expectations,42.9,2025-07-01,41.8,1.1,0.54,1.2,0.80,2025-11-14 16:26:00


In [13]:
consumer_confidence_collection.dashboard.plots(hlines=[100, 50])

##### Business Survey

In [14]:
business_surveys.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Industrial Enterprises Macro Performance,27.1,2025-07-01,26.0,1.1,0.22,-0.20,0.20,2025-11-14 16:26:00
Industrial Enterprises Overseas Orders,46.4,2025-07-01,44.3,2.1,0.41,6.7,-0.20,2025-11-14 16:26:00
Logistics Prosperity Index,50.7,2025-10-01,51.2,-0.50,-0.14,-0.40,-1.9,2025-11-14 16:26:00
Economic Policy Uncertainty (News-Based),522.4,2025-10-01,221.8,300.6,4.4,-2.4,294.3,2025-11-14 16:26:00
Trade Policy Uncertainty,685.7,2025-10-01,400.6,285.1,1.7,-512.0,583.7,2025-11-14 16:26:00
Yicai Chief Economists Confidence,50.3,2025-11-01,50.3,0.00,0.00,0.46,-0.48,2025-11-14 16:26:00
CKGSB Business Conditions Index,52.0,2025-10-01,51.1,0.90,0.21,1.9,3.9,2025-11-14 16:26:00
Small & Medium Enterprises Development Index,89.1,2025-04-01,89.5,-0.40,-0.22,0.10,0.10,2025-11-14 16:26:00


In [34]:
business_surveys.dashboard.plots()

##### Cycle Indexes

In [33]:
climate_indexes_df = climate_indexes.df
climate_indexes_df.columns = list(climate_indexes.id_to_title.values())
fig = plot_lines(
    climate_indexes_df,
    title="<b>Macroeconomic Climate Index</b>",
    source="National Bureau of Statistics (NBS) of China",
)
fig.add_hline(y=100, line_width=2, line_dash="dash", line_color="black")
fig.show()

#### Industrial Production & Fixed Asset Investment

**TODO**: Key growth indicators
- Industrial Production YoY
- Fixed Asset Investment YoY (total and ex-rural)
- Retail Sales YoY
- Manufacturing Investment
- Infrastructure Investment

### GDP Growth & Components

**TODO**: GDP growth metrics
- Real GDP YoY and QoQ
- Nominal GDP growth
- GDP components: Consumption, Investment, Net Exports
- Consider using Haver series with `yryr%()` transformation for YoY calculations

In [37]:
gdp_by_supply = hv.create_collection(
    {
        "Gross domestic product": "H924NGPC@EMERGEPR",
        "Primary industry": "H924G1C@EMERGEPR",
        "Secondary industry": "H924G2C@EMERGEPR",
        "Tertiary industry (services)": "H924G3C@EMERGEPR",
    }
)

In [38]:
gdp_by_supply.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Gross domestic product,"33,772",2025-07-01,"33,474",298.5,0.79,647.1,"1,565",2025-11-14 16:47:00
Primary industry,"2,441",2025-07-01,"2,415",25.7,1.5,60.5,93.6,2025-11-14 16:47:00
Secondary industry,"12,150",2025-07-01,"12,097",53.6,0.27,69.2,478.0,2025-11-14 16:47:00
Tertiary industry (services),"19,181",2025-07-01,"18,962",219.3,1.2,517.3,993.7,2025-11-14 16:47:00


In [ ]:
gdp_stats = {
    "Real GDP YoY": "yryr%(...)",
    "Consumption": "yryr%(...)",
    "Fixed Investment": "yryr%(...)",
    "Exports": "yryr%(...)",
}

### Inflation Measures

**TODO**: CPI and PPI inflation
- CPI Headline YoY
- CPI Core (ex food & energy)
- CPI Food component (important for China)
- PPI YoY (Producer Price Index - useful for deflation signals)
- Consider MoM changes as well
- Use `hlines=` in plots to show target levels if applicable

In [ ]:
# inflation = {
#     "CPI YoY": "yryr%(...)",
#     "CPI Core YoY": "yryr%(...)",
#     "CPI Food YoY": "yryr%(...)",
#     "PPI YoY": "yryr%(...)",
# }
# inflation_collection = hc.create_collection(inflation)

### Labor Market & Slack Measures

**TODO**: Employment and slack indicators
- Surveyed Unemployment Rate (urban)
- Youth Unemployment (if available - note recent data suspension)
- Manufacturing employment indices from PMI surveys
- Wage growth indicators
- Note: Set `good_is = -1` for unemployment metrics

In [ ]:
# employment = {
#     "Surveyed Unemployment Rate": "...",
#     "Manufacturing Employment PMI": "...",
# }
# employment_collection = hc.create_collection(employment)
# employment_collection[0].good_is = -1

#### Output Gap

**TODO**: If available, add output gap estimates
- Check for China output gap series in Haver or Bloomberg
- Use `show_0=True` in plot to emphasize zero line

In [ ]:
# output_gap = hc.get_series("...")
# output_gap.plot(show_0=True, tick_suffix="%")

### Trade & External Balance

**TODO**: Trade statistics - critical for China
- Exports YoY (USD and local currency terms)
- Imports YoY
- Trade Balance
- Current Account Balance % GDP
- FX Reserves
- Consider breaking down exports by destination (US, EU, etc.) if available

In [ ]:
# trade_stats = {
#     "Exports YoY USD": "...",
#     "Imports YoY USD": "...",
#     "Trade Balance": "...",
#     "Current Account % GDP": "...",
# }
# trade_collection = hc.create_collection(trade_stats)

### Credit Creation & Financial Conditions

**TODO**: Credit aggregates - crucial for China macro
- Total Social Financing (TSF) - broadest credit measure
- New Yuan Loans
- M2 Money Supply growth
- Shadow banking components (trust loans, entrusted loans)
- Corporate debt % GDP
- Household debt % GDP
- Local Government Financing Vehicle (LGFV) debt if available

In [ ]:
# credit_stats = {
#     "Total Social Financing YoY": "...",
#     "New Yuan Loans": "...",
#     "M2 YoY": "...",
#     "Corporate Debt % GDP": "...",
#     "Household Debt % GDP": "...",
# }
# credit_collection = hc.create_collection(credit_stats)

### Property Sector Indicators

**TODO**: Real estate metrics - systemically important for China
- New home prices (70-city index)
- Property sales (floor space sold)
- Property investment YoY
- Housing starts
- Developer funding sources
- Inventory levels (months to clear)

In [ ]:
# property_stats = {
#     "New Home Prices 70-city": "...",
#     "Property Sales YoY": "...",
#     "Property Investment YoY": "...",
# }
# property_collection = hc.create_collection(property_stats)

### Fiscal Policy & Government Budget

**TODO**: Government finance
- Fiscal balance (augmented deficit including off-budget)
- Government debt % GDP
- Local government debt
- Fiscal revenue and expenditure growth

In [ ]:
# fiscal_stats = {
#     "Fiscal Balance % GDP": "...",
#     "Government Debt % GDP": "...",
# }

### Business & Consumer Surveys

**TODO**: Sentiment indicators
- Consumer confidence index
- Business confidence surveys
- Expectations components from PMI surveys
- Use `good_is = 1` for confidence indices, set appropriate `hlines=` for neutral levels (often 50 or 100)

In [ ]:
# surveys = {
#     "Consumer Confidence": "...",
#     "Business Climate Index": "...",
# }

### Additional China-Specific Indicators

**TODO**: China-specific metrics to consider:
- Li Keqiang Index (electricity consumption, rail freight, loans)
- Commodity imports (iron ore, copper - proxy for construction activity)
- Car sales
- Semiconductor sales
- Tech sector indicators
- Capital flows (FDI, portfolio flows)
- RMB exchange rate and volatility

In [ ]:
# china_specific = {
#     "Electricity Production YoY": "...",
#     "Rail Freight YoY": "...",
#     "Auto Sales YoY": "...",
# }

### AI-Powered Analysis

**TODO**: Use `iris.summarize()` on key collections to get AI-generated insights
- Example: `iris.summarize(credit_collection, "What trends do you see in China's credit creation?")`
- Can also reference external sources: `iris.summarize(collection, "Summary with context from: https://...")`

In [ ]:
# iris_analysis = iris.summarize(some_collection, "What trends are you seeing?")
# iris_analysis.html()

---

In [ ]:
Markdown(f"_Notebook updated at {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}_")